In [9]:
from __future__ import print_function

import pandas as pd
import numpy as np

from src import tracker
from genepy import terra
from genepy.utils import helper as h
from genepy.google import gcp
from gsheets import Sheets
from taigapy import TaigaClient
import dalmatian as dm
from genepy.google.google_sheet import dfToSheet

from IPython.display import Image,display
from bokeh.plotting import *

%load_ext autoreload
%autoreload 2
%load_ext rpy2.ipython
tc = TaigaClient()
output_notebook()

my_id = '~/.client_secret.json'
mystorage_id = "~/.storage.json"
# do the first steps of https://medium.com/craftsmenltd/from-csv-to-google-sheet-using-python-ef097cb014f9
creds = '../.credentials.json'

sheets = Sheets.from_files(my_id, mystorage_id)
replace = {'T': 'Tumor', 'N': 'Normal', 'm': 'Unknown', 'L': 'Unknown'}

/home/jeremie/miniconda3/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex


Loading BokehJS ...

## boot up

we are instanciating all the parameters needed for this pipeline to run

In [10]:
samplesetname = "21Q1"

workspace1="terra-broad-cancer-prod/DepMap_WGS"
workspace2="terra-broad-cancer-prod/Getz_IBM_CellLines_WGS"

refworkspace="broad-firecloud-ccle/DepMap_WGS_CN"
cgaworkspace="broad-firecloud-ccle/DepMap_Mutation_Calling_CGA_pipeline-wgs"

source1="ccle"
source2="ibm"

refsheet_url = "https://docs.google.com/spreadsheets/d/1Pgb5fIClGnErEqzxpU7qqX6ULpGTDjvzWwDN8XUJKIY"
sheeturl = "https://docs.google.com/spreadsheets/d/115TUgA1t_mD32SnWAGpW9OKmJ2W5WYAOs3SuSdedpX4"
potential_list_url = "https://docs.google.com/spreadsheets/d/1BEgH03V4OmGhYeciLCZV00h6hp3WkO0basahS93akCE"

depmap_pv = "https://docs.google.com/spreadsheets/d/1uqCOos-T9EMQU7y2ZUw4Nm84opU5fIT1y7jet1vnScE"

release = samplesetname

WGSmethods = [
    "gatk/PreProcessingForVariantDiscovery_GATK4/8",
    "GP-TAG/Manta_SomaticSV/9",
    "gkugener/ArrayOfFilesToTxt/1",
    "vdauwera/BamToUnmappedRGBams/4",
    "gatk/CNV_Somatic_Pair_Workflow/9",
    "gkugener/Aggregate_CN_seg_files/2"
]

maxage='2020-09-10'

# version 102
ensemblserver = "http://nov2020.archive.ensembl.org/biomart" 

In [11]:
wm1 = dm.WorkspaceManager(workspace1)
wm2 = dm.WorkspaceManager(workspace2)

refwm = dm.WorkspaceManager(refworkspace).disable_hound()
cgawm = dm.WorkspaceManager(cgaworkspace).disable_hound()

/home/jeremie/miniconda3/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/home/jeremie/miniconda3/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about servic

In [12]:
extract_to_change = {'from_arxspan_id': 'participant'}
potential_list = sheets.get(potential_list_url).sheets[0].to_frame().values.T[0].tolist()
ccle_refsamples = sheets.get(refsheet_url).sheets[0].to_frame(index_col=0)
#ccle_refsamples=pd.read_csv('temp/updated_ref_samples.csv',index_col=0)

depmap_pv = sheets.get(depmap_pv).sheets[0].to_frame(header=2)
depmap_pv = depmap_pv.drop(depmap_pv.iloc[:1].index).set_index('DepMap_ID', drop=True)[['Culture Type', 'Culture Medium']]
depmap_master = tc.get(name='arxspan-cell-line-export-f808', file='ACH').set_index('arxspan_id',drop=True)
depmap_master = depmap_master.join(depmap_pv)

[##################]100% |  23.5 MiB/s |   1.0 MiB /   1.0 MiB | Time:  0:00:00


In [9]:
wm = dm.WorkspaceManager("broad-firecloud-ccle/DepMap_hg38_RNAseq")
a=ccle_refsamples.loc[["CDS-G0F5f5", "CDS-kU30H5", "CDS-fk564T"], ["stripped_cell_name", "participant_id", "cellosaurus_id", "internal_bam_filepath", "internal_bai_filepath", "legacy_bam_filepath", "legacy_bai_filepath", "parent_cell_line", "sex", "matched_normal", "age", "primary_site", "primary_disease", "subtype", "subsubtype", "origin", "mediatype", "condition", "baits", "source", "sequencing_date", 'size', 'crc32c_hash', 'md5_hash', 'legacy_size', "legacy_crc32c_hash"]]
a.index.name="sample_id"
a = a.rename(columns={'patient_id': 'participant_id'})
wm.upload_samples(a)

Successfully imported 3 participants.
Successfully imported 3 samples.


In [10]:
wm.update_sample_set('toremap', a.index.tolist())

Successfully imported 1 sample sets:
  * toremap (3 samples)


In [14]:
wm.create_submission('star_v1-0_BETA_cfg', "toremap", "sample_set", "this.samples")

Successfully created submission 110b9f37-9430-40ef-ac0a-9c7daf60b898.


'110b9f37-9430-40ef-ac0a-9c7daf60b898'

In [15]:
sam= wm.get_samples()

In [17]:
sam.loc[["CDS-G0F5f5", "CDS-kU30H5", "CDS-fk564T"], ['star_bam_file', 'star_bam_index']]

,star_bam_file,star_bam_index
sample_id,,
CDS-G0F5f5,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...
CDS-kU30H5,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...
CDS-fk564T,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...,gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd5004...


In [18]:
a=terra.changeToBucket(sam.loc[["CDS-G0F5f5", "CDS-kU30H5", "CDS-fk564T"], ['star_bam_file', 'star_bam_index']],'gs://cclebams/rnasq_hg38/', name_col= "index" , values=['star_bam_file','star_bam_index'], filetypes=['bam', 'bai'], catchdup=True, test=False)

In [26]:
ccle_refsamples.loc[["CDS-G0F5f5", "CDS-kU30H5", "CDS-fk564T"], ['internal_bam_filepath', 'internal_bai_filepath']]

,internal_bam_filepath,internal_bai_filepath
CDS-G0F5f5,gs://cclebams/rnasq_hg38/CDS-G0F5f5.bam,gs://cclebams/rnasq_hg38/CDS-G0F5f5.bai
CDS-kU30H5,gs://cclebams/rnasq_hg38/CDS-kU30H5.bam,gs://cclebams/rnasq_hg38/CDS-kU30H5.bai
CDS-fk564T,gs://cclebams/rnasq_hg38/CDS-fk564T.bam,gs://cclebams/rnasq_hg38/CDS-fk564T.bai


In [27]:
dfToSheet(ccle_refsamples,'ccle sample tracker', secret=creds)

In [193]:
a = ! gsutil ls gs://cclebams/rnasq_hg38/

In [201]:
set(ccle_refsamples[ccle_refsamples.datatype.isin(['rna'])].index) - set(["CDS"+i.split('/CDS')[-1].split('.Ali')[0] for i in a])

{'CDS-ABH0uZ', 'CDS-G0F5f5', 'CDS-fk564T', 'CDS-kU30H5'}

In [183]:
l

['gs://cclebams/rna/921.bai',
 'gs://cclebams/rna/921.bam',
 'gs://cclebams/rna/94T778.bai',
 'gs://cclebams/rna/95T1000.bai',
 'gs://cclebams/rna/95T1000.bam',
 'gs://cclebams/rna/95T1000.bam.md5',
 'gs://cclebams/rna/A375SKINCJ1.bai',
 'gs://cclebams/rna/A375SKINCJ1.bam.md5',
 'gs://cclebams/rna/A375SKINCJ2.bai',
 'gs://cclebams/rna/A375SKINCJ2.bam',
 'gs://cclebams/rna/A375SKINCJ2.bam.md5',
 'gs://cclebams/rna/A375SKINCJ3.bai',
 'gs://cclebams/rna/A375SKINCJ3.bam.md5',
 'gs://cclebams/rna/A388.bai',
 'gs://cclebams/rna/ACxtgE_M040416.bai',
 'gs://cclebams/rna/ACxtgE_M040416.bam',
 'gs://cclebams/rna/AfBxpu_RMSYM.bai',
 'gs://cclebams/rna/AfBxpu_RMSYM.bam',
 'gs://cclebams/rna/BPH1.bai',
 'gs://cclebams/rna/Becker.bai',
 'gs://cclebams/rna/C396.bai',
 'gs://cclebams/rna/C396.bam.md5',
 'gs://cclebams/rna/CASKI.bai',
 'gs://cclebams/rna/CCLFUPGI0005T.bai',
 'gs://cclebams/rna/CCLP1.bai',
 'gs://cclebams/rna/CCLP1.bam.md5',
 'gs://cclebams/rna/CCSW1.bai',
 'gs://cclebams/rna/CCSW1.bam'

In [504]:
i  = ! gsutil ls gs://cclebams/**

In [449]:
b = [a for a in i if a.endswith('.bam')]

In [469]:
len(b)

2122

In [494]:
bi = [a for a in i if a.endswith('.bam') and a[:-4]+".bai" not in i and a+".bai" not in i]

In [497]:
len(bi)

41

In [511]:
! gsutil ls gs://cclebams/wes/Getz_IBM_DMX_CellLines_12samples_deepcov_01302018/RP-1561/Exome/RVH421SKINFV1/v1/RVH421SKINFV1.bam

CommandException: One or more URLs matched no objects.


In [513]:
u = set(ccle_refsamples[ccle_refsamples.datatype=="wes"].legacy_bam_filepath)- set([a for a in i if a.endswith('.bam')])

In [526]:
sizes = [gcp.extractSize(a) for a in gcp.lsFiles(set([a for a in i if a.endswith('.bam')]) - set(ccle_refsamples[ccle_refsamples.datatype=="wes"].legacy_bam_filepath), "-l")]

listing files in gs


In [527]:
sizes

[('gs://cclebams/wes/RP-1500/Exome/ESO51_1/current/ESO51_1.bam', 26288709027),
 ('gs://cclebams/wes/CDS-2JxT1P.bam', 17135716554),
 ('gs://cclebams/wes/CDS-OLgoE4.bam', 31217641284),
 ('gs://cclebams/wes/CDS-Ql8GJZ.bam', 27878740151),
 ('gs://cclebams/wes/CDS-wlTAAF.bam', 25252308262),
 ('gs://cclebams/wes/CDS-lTogDX.bam', 26167931603),
 ('gs://cclebams/wes/CDS-9zidMf.bam', 10570660641),
 ('gs://cclebams/wes/CDS-NPG23x.bam', 21689262653),
 ('gs://cclebams/wes/CDS-6EyvRQ.bam', 27725567504),
 ('gs://cclebams/wes/RP-1561/Exome/SK-MEL-2/current/SK-MEL-2.bam',
  28757788344),
 ('gs://cclebams/wes/CDS-PHI8VT.bam', 22496779642),
 ('gs://cclebams/wes/CDS-Y27yfi.bam', 26154087701),
 ('gs://cclebams/wes/CDS-Pkk9e2.bam', 23213429069),
 ('gs://cclebams/wes/CDS-IzeN7a.bam', 22051020828),
 ('gs://cclebams/wes/CDS-Md89va.bam', 21788262259),
 ('gs://cclebams/wes/CDS-ohjYlg.bam', 23932006016),
 ('gs://cclebams/wes/CDS-G1sVsw.bam', 17999530720),
 ('gs://cclebams/wes/CDS-wbPtTZ.bam', 26597306271),
 ('gs:

In [532]:
torm = []
tofind = []
for val in sizes:
    a = ccle_refsamples[ccle_refsamples.legacy_size==val[1]]
    if len(a)>0:
        torm.append(val[0])
        continue
    a = ccle_refsamples[ccle_refsamples['size']==val[1]]
    if len(a)>0:
        torm.append(val[0])
        continue
    tofind.append(val[0])

In [535]:
! gsutil ls -l gs://cclebams/wes/RP-1500/Exome/ESO51_1/current/ESO51_1.bam

26288709027  2020-03-09T15:43:34Z  gs://cclebams/wes/RP-1500/Exome/ESO51_1/current/ESO51_1.bam
TOTAL: 1 objects, 26288709027 bytes (24.48 GiB)


In [537]:
! gsutil ls -l gs://cclebams/wes/CDS-CmkOx5.bam

24761705198  2020-09-24T14:22:02Z  gs://cclebams/wes/CDS-CmkOx5.bam
TOTAL: 1 objects, 24761705198 bytes (23.06 GiB)


In [539]:
! gsutil ls -l gs://cclebams/wes/Getz_IBM_CellLine_WES_April2019/RP-1561/Exome/BPH1/v1/BPH1.bam

24761705198  2020-03-09T15:42:51Z  gs://cclebams/wes/Getz_IBM_CellLine_WES_April2019/RP-1561/Exome/BPH1/v1/BPH1.bam
TOTAL: 1 objects, 24761705198 bytes (23.06 GiB)


In [5]:
## missing legacy for rna/wes
ccle_refsamples[ccle_refsamples.datatype.isin(['rna','wes'])& ccle_refsamples.legacy_bam_filepath.isna()]

,arxspan_id,version,sm_id,PDO,datatype,size,ccle_name,stripped_cell_name,participant_id,cellosaurus_id,...,19q2,19q3,19q4,20q1,20q2,20q4,20q,21Q1,low_quality,legacy_crc32_hash


In [6]:
ccle_refsamples[ccle_refsamples.datatype.isin(['rna','wes'])& ccle_refsamples.legacy_bai_filepath.isna()]

,arxspan_id,version,sm_id,PDO,datatype,size,ccle_name,stripped_cell_name,participant_id,cellosaurus_id,...,19q2,19q3,19q4,20q1,20q2,20q4,20q,21Q1,low_quality,legacy_crc32_hash


In [6]:
## wrong filepath legacy for rna/wes
miss_rwbi = gcp.exists(ccle_refsamples[ccle_refsamples.datatype.isin(['rna','wes'])].legacy_bai_filepath.tolist())

listing files in gs


In [7]:
miss_rw = gcp.exists(ccle_refsamples[ccle_refsamples.datatype.isin(['rna','wes'])].legacy_bam_filepath.tolist())

listing files in gs


In [8]:
## which ones can we get from remapping
remapable = ccle_refsamples[ccle_refsamples.legacy_bam_filepath.isin(miss_rw[1]) & ~ccle_refsamples.internal_bam_filepath.isna()]
len(remapable)

156

In [12]:
remapable[remapable.datatype=='wes']

,arxspan_id,version,sm_id,PDO,datatype,size,ccle_name,stripped_cell_name,participant_id,cellosaurus_id,...,19q2,19q3,19q4,20q1,20q2,20q4,20q,21Q1,low_quality,legacy_crc32_hash
CDS-fjMmZs,ACH-001529,2,NaN,NaN,wes,1.925572e+10,NaN,JAR,PT-4VYF6QwQ,CVCL_0360,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
CDS-i7LdYv,ACH-000051,1,NaN,NaN,wes,1.061912e+10,NaN,TE617T,PT-2idRHKaT,CVCL_1755,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
CDS-M33guL,ACH-000032,1,NaN,NaN,wes,4.340937e+09,NaN,MHHCALL3,PT-jgh3Ohro,CVCL_0089,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
CDS-tfg4Dr,ACH-001282,1,NaN,NaN,wes,7.107219e+09,NaN,CHLA258,PT-PiD3Gm5d,CVCL_A058,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
CDS-CAB69b,ACH-000728,1,NaN,NaN,wes,4.291779e+09,NaN,KASUMI2,PT-MS6R4JQq,CVCL_0590,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CDS-clTgER,ACH-000577,2,NaN,NaN,wes,2.511173e+10,NaN,JHH2,PT-R2lQDrhy,CVCL_2786,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
CDS-wXWgLf,ACH-000596,2,NaN,NaN,wes,1.983989e+10,NaN,LCLC97TM1,PT-MgrYS90q,CVCL_1376,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
CDS-GTkHGf,ACH-001075,2,NaN,NaN,wes,4.387466e+10,NaN,NCIH292,PT-0IujQn8b,CVCL_0455,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
CDS-COhPnV,ACH-000800,2,NaN,NaN,wes,6.522894e+10,NaN,NCIH446,PT-bRUartb7,CVCL_1562,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN


In [31]:
## remapping
weswm = dm.WorkspaceManager("broad-firecloud-ccle/DepMap_WES_CN_hg38")

weswm.update_sample_set("toremap", remapable.index.tolist())

In [19]:
a = weswm.create_submission('BamToUnmappedRGBams_MC', "toremap", "sample_set", "this.samples")

Successfully created submission cdd5a065-d28d-4750-9417-efae23e5e893.


TypeError: waitForSubmission() missing 1 required positional argument: 'submissions'

In [32]:
a = weswm.create_submission('Generate_uBAM_File_List', "toremap", "sample_set", "this.samples")

/home/jeremie/miniconda3/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Successfully created submission 34b65cef-361e-420f-b323-94cc52ae0170.


/home/jeremie/miniconda3/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/home/jeremie/miniconda3/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about servic

In [73]:
a = weswm.create_submission('Realign_WES_GATK4_hg19', "toremap", "sample_set", "this.samples")

Successfully created submission 61d87541-d51b-4de0-b9ac-847ce2106d08.


In [557]:
## the ones we can't
nonremapable = ccle_refsamples[ccle_refsamples.legacy_bam_filepath.isin(miss_rw[1]) & ccle_refsamples.internal_bam_filepath.isna()]
nonremapable

,arxspan_id,version,sm_id,PDO,datatype,size,ccle_name,stripped_cell_name,participant_id,cellosaurus_id,...,19q2,19q3,19q4,20q1,20q2,20q4,20q,21Q1,low_quality,legacy_crc32_hash
CDS-d2xe4x,ACH-000580,1,NaN,NaN,wes,7.088072e+09,NaN,C32,PT-IBx5yvTA,CVCL_1097,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
CDS-ezLdbO,ACH-001090,1,NaN,NaN,wes,8.618654e+09,NaN,HN,PT-I51jWv5J,CVCL_1283,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
CDS-NE61ys,ACH-000282,1,NaN,NaN,wes,6.064032e+09,NaN,NCIH1755,PT-h8sjmlVl,CVCL_1492,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN


In [ ]:
## indexing
missrwbi

In [ ]:
## recompute size/ hashes for rna/wes legacy

In [11]:
## missing bam filepath 
missing = ccle_refsamples[ccle_refsamples.internal_bam_filepath.isna()]
missing

,arxspan_id,version,sm_id,PDO,datatype,size,ccle_name,stripped_cell_name,participant_id,cellosaurus_id,...,19q2,19q3,19q4,20q1,20q2,20q4,20q,21Q1,low_quality,legacy_crc32_hash
CDS-d2xe4x,ACH-000580,1,NaN,NaN,wes,7.088072e+09,NaN,C32,PT-IBx5yvTA,CVCL_1097,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
CDS-ezLdbO,ACH-001090,1,NaN,NaN,wes,8.618654e+09,NaN,HN,PT-I51jWv5J,CVCL_1283,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
CDS-NE61ys,ACH-000282,1,NaN,NaN,wes,6.064032e+09,NaN,NCIH1755,PT-h8sjmlVl,CVCL_1492,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN


In [60]:
## missing bai filepath
missing = ccle_refsamples[ccle_refsamples.internal_bai_filepath.isna()]
missing

,arxspan_id,version,sm_id,PDO,datatype,size,ccle_name,stripped_cell_name,participant_id,cellosaurus_id,...,19q2,19q3,19q4,20q1,20q2,20q4,20q,21Q1,low_quality,legacy_crc32_hash
CDS-d2xe4x,ACH-000580,1,NaN,NaN,wes,7.088072e+09,NaN,C32,PT-IBx5yvTA,CVCL_1097,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
CDS-ezLdbO,ACH-001090,1,NaN,NaN,wes,8.618654e+09,NaN,HN,PT-I51jWv5J,CVCL_1283,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
CDS-NE61ys,ACH-000282,1,NaN,NaN,wes,6.064032e+09,NaN,NCIH1755,PT-h8sjmlVl,CVCL_1492,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN


In [562]:
## wrong bam filepath
wrongfilepath = gcp.exists(ccle_refsamples[~ccle_refsamples.internal_bam_filepath.isna()].internal_bam_filepath.tolist())

listing files in gs


In [565]:
wrongfilepath

(True, set())

In [13]:
## recompute size/hashes for bam filepath
ccle_refsamples.loc[ccle_refsamples[~ccle_refsamples.internal_bam_filepath.isna()].index, "crc32c_hash"] = [gcp.extractHash(i) for i in gcp.lsFiles(ccle_refsamples[~ccle_refsamples.internal_bam_filepath.isna()].internal_bam_filepath, "-L")]
ccle_refsamples.loc[ccle_refsamples[~ccle_refsamples.internal_bam_filepath.isna()].index, "size"] = [gcp.extractSize(i)[1] for i in gcp.lsFiles(ccle_refsamples[~ccle_refsamples.internal_bam_filepath.isna()].internal_bam_filepath, "-l")]
ccle_refsamples.loc[ccle_refsamples[~ccle_refsamples.internal_bam_filepath.isna()].index, "md5_hash"] = [gcp.extractHash(i, "md5") for i in gcp.lsFiles(ccle_refsamples[~ccle_refsamples.internal_bam_filepath.isna()].internal_bam_filepath, "-L")]

listing files in gs
listing files in gs
listing files in gs


In [1]:
## duplicate size accross legacy/internal 
a

NameError: name 'a' is not defined

In [14]:
## missingbam index
wrongfilepath = gcp.exists(ccle_refsamples[~ccle_refsamples.internal_bai_filepath.isna()].internal_bai_filepath.tolist())

listing files in gs


In [24]:
## bam index non matching anything
bais= await seq.indexBams(bams= ccle_refsamples[ccle_refsamples.internal_bai_filepath.isin(wrongfilepath[1])].internal_bam_filepath, bucketpath=None, cores=10)

In [27]:
for k,val in bais.items():
    ccle_refsamples.loc[ccle_refsamples[ccle_refsamples.internal_bam_filepath == k].index, 'internal_bai_filepath'] = val

In [29]:
dfToSheet(ccle_refsamples,'ccle sample tracker', secret=creds)

In [33]:
i  = ! gsutil ls gs://cclebams/**

In [39]:
## bam non matching anything in the sample tracker
nonmatch = set([a for a in i if a.endswith('.bam')]) - set(ccle_refsamples.internal_bam_filepath.tolist()+ ccle_refsamples.legacy_bam_filepath.tolist())
nonmatch

{'gs://cclebams/hg38_wes/ACH-001421_2.hg38.bam',
 'gs://cclebams/hg38_wes/ACH-001533_1.hg38.bam',
 'gs://cclebams/hg38_wes/ACH-001574_1.hg38.bam',
 'gs://cclebams/hg38_wes/ACH-001678_1.hg38.bam',
 'gs://cclebams/hg38_wes/ACH-001711_3.hg38.bam',
 'gs://cclebams/hg38_wes/ACH-001847_1.hg38.bam',
 'gs://cclebams/hg38_wes/ACH-002021_1.hg38.bam',
 'gs://cclebams/hg38_wes/ACH-002048_1.hg38.bam',
 'gs://cclebams/hg38_wes/ACH-002065_1.hg38.bam',
 'gs://cclebams/hg38_wes/ACH-003000_1.hg38.bam',
 'gs://cclebams/hg38_wes/CDS-0aJ4Yh.hg38.bam',
 'gs://cclebams/hg38_wes/CDS-0lfqVz.hg38.bam',
 'gs://cclebams/hg38_wes/CDS-0pZb0j.hg38.bam',
 'gs://cclebams/hg38_wes/CDS-0qPmaJ.hg38.bam',
 'gs://cclebams/hg38_wes/CDS-1PXzlf.hg38.bam',
 'gs://cclebams/hg38_wes/CDS-1b1Hxk.hg38.bam',
 'gs://cclebams/hg38_wes/CDS-1djAlo.hg38.bam',
 'gs://cclebams/hg38_wes/CDS-1p2nnc.hg38.bam',
 'gs://cclebams/hg38_wes/CDS-1uWUTi.hg38.bam',
 'gs://cclebams/hg38_wes/CDS-294bk6.hg38.bam',
 'gs://cclebams/hg38_wes/CDS-2JxT1P.hg38

In [44]:
## bam that match something in size and can be deleted or can replace the other one
sizze=[gcp.extractSize(i) for i in gcp.lsFiles(nonmatch, '-l')]

listing files in gs


In [46]:
sizze

[('gs://cclebams/hg38_wes/CDS-ydPJEM.hg38.bam', 33590349704),
 ('gs://cclebams/hg38_wes/CDS-s7pOQR.hg38.bam', 52259732517),
 ('gs://cclebams/wes/CDS-eGQYXr.bam', 24098037187),
 ('gs://cclebams/hg38_wes/CDS-Pkk9e2.hg38.bam', 16985791742),
 ('gs://cclebams/wes/CDS-Fz0HXE.bam', 25843558785),
 ('gs://cclebams/hg38_wes/CDS-soTPPi.hg38.bam', 15724674947),
 ('gs://cclebams/hg38_wes/CDS-iKXYuH.hg38.bam', 18181200052),
 ('gs://cclebams/wes/CDS-4S6juQ.bam', 29576566390),
 ('gs://cclebams/wes/CDS-UL1jLm.bam', 25516864764),
 ('gs://cclebams/rna/UPCISCC111.bam', 3821164738),
 ('gs://cclebams/hg38_wes/CDS-KgRznV.hg38.bam', 16089050631),
 ('gs://cclebams/hg38_wes/CDS-txTRwz.hg38.bam', 13238732111),
 ('gs://cclebams/wes/RP-1500/Exome/ESO26_1/current/ESO26_1.bam', 22776809217),
 ('gs://cclebams/hg38_wes/CDS-zwAn7G.hg38.bam', 20340862682),
 ('gs://cclebams/hg38_wes/CDS-fs8moU.hg38.bam', 21098867720),
 ('gs://cclebams/wes/CDS-gKIdjs.bam', 16134677521),
 ('gs://cclebams/hg38_wes/CDS-kAARUi.hg38.bam', 1877

In [52]:
miss_rw

(False,
 {'gs://cclebams/wes/C817/CHLA-258/current/CHLA-258.bam',
  'gs://cclebams/wes/C836.253J.1.bam',
  'gs://cclebams/wes/C836.GDM-1.2.bam',
  'gs://cclebams/wes/C836.GRANTA-519.2.bam',
  'gs://cclebams/wes/C836.HMC-1-8.2.bam',
  'gs://cclebams/wes/C836.JJN-3.1.bam',
  'gs://cclebams/wes/C836.JMSU-1.1.bam',
  'gs://cclebams/wes/C836.KASUMI-2.1.bam',
  'gs://cclebams/wes/C836.KMM-1.1.bam',
  'gs://cclebams/wes/C836.KMS-12-BM.1.bam',
  'gs://cclebams/wes/C836.KMS-20.1.bam',
  'gs://cclebams/wes/C836.L3.3.1.bam',
  'gs://cclebams/wes/C836.LMSU.2.bam',
  'gs://cclebams/wes/C836.MFE-296.2.bam',
  'gs://cclebams/wes/C836.MHH-CALL-2.1.bam',
  'gs://cclebams/wes/C836.MHH-CALL-3.1.bam',
  'gs://cclebams/wes/C836.NCI-H1341.2.bam',
  'gs://cclebams/wes/C836.NCI-H1373.1.bam',
  'gs://cclebams/wes/C836.OVTOKO.2.bam',
  'gs://cclebams/wes/C836.PLC_PRF_5.2.bam',
  'gs://cclebams/wes/C836.RD-ES.2.bam',
  'gs://cclebams/wes/C836.SNU-175.2.bam',
  'gs://cclebams/wes/C836.T3M-4.2.bam',
  'gs://ccleba

In [68]:
found=[]
torm=[]
remaining = set()
for val in sizze:
    res= ccle_refsamples[(ccle_refsamples['legacy_size']==val[1])]
    if len(res) >0:
        if res['legacy_bam_filepath'].values[0] in miss_rw[1]:
            print("found!-----------")
            ccle_refsamples.loc[res.index, 'legacy_bam_filepath'] = val[0]
            found.append(res.index[0])
        else:
            torm.append(val[0])
    else:
        remaining.add(val[0])

found!-----------
found!-----------
found!-----------
found!-----------
found!-----------
found!-----------
found!-----------
found!-----------
found!-----------
found!-----------
found!-----------
found!-----------
found!-----------
found!-----------
found!-----------
found!-----------
found!-----------
found!-----------
found!-----------
found!-----------


In [69]:
found

['CDS-C95iHs',
 'CDS-qrjNBy',
 'CDS-ixXfW6',
 'CDS-TOM1NB',
 'CDS-pQE1r4',
 'CDS-7oAFaO',
 'CDS-MQDYm1',
 'CDS-FNFqnr',
 'CDS-elPhWo',
 'CDS-rG42Rn',
 'CDS-C9v2MI',
 'CDS-Oidz6E',
 'CDS-1xwlXO',
 'CDS-uMD8jg',
 'CDS-XL3EP1',
 'CDS-oLN6S3',
 'CDS-fxAhOu',
 'CDS-q0PJc9',
 'CDS-kpEDhO',
 'CDS-TsX9vB']

In [70]:
torm

['gs://cclebams/wes/CDS-eGQYXr.bam',
 'gs://cclebams/wes/CDS-Fz0HXE.bam',
 'gs://cclebams/wes/CDS-4S6juQ.bam',
 'gs://cclebams/wes/CDS-UL1jLm.bam',
 'gs://cclebams/rna/UPCISCC111.bam',
 'gs://cclebams/wes/CDS-gKIdjs.bam',
 'gs://cclebams/rna/TERA2.bam',
 'gs://cclebams/wes/CDS-Hx6zuD.bam',
 'gs://cclebams/wes/CDS-Xgu4mi.bam',
 'gs://cclebams/wes/CDS-kt2Gne.bam',
 'gs://cclebams/wes/CDS-SvzhGj.bam',
 'gs://cclebams/wes/CDS-Pku96X.bam',
 'gs://cclebams/wes/CDS-leyYAD.bam',
 'gs://cclebams/wes/CDS-R6ehaT.bam',
 'gs://cclebams/wes/CDS-cKMeDY.bam',
 'gs://cclebams/wes/CDS-I7bMcd.bam',
 'gs://cclebams/wes/CDS-GnBdHN.bam',
 'gs://cclebams/wes/CDS-PHI8VT.bam',
 'gs://cclebams/wes/CDS-Y27yfi.bam',
 'gs://cclebams/wes/CDS-sieIuO.bam',
 'gs://cclebams/rna/MM253.bam',
 'gs://cclebams/wes/CDS-3mvYnW.bam',
 'gs://cclebams/rna/PEDS117.bam',
 'gs://cclebams/wes/CDS-AOWMF3.bam',
 'gs://cclebams/wes/CDS-8z476r.bam',
 'gs://cclebams/wes/CDS-qaOoHQ.bam',
 'gs://cclebams/wes/CDS-xCyamv.bam',
 'gs://cclebam

In [71]:
for val in sizze:
    res= ccle_refsamples[(ccle_refsamples['size']==val[1])]
    if len(res) >0:
        print(val[0], res['internal_bam_filepath'])
        torm.append(val[0])
        remaining.remove(val[0])

gs://cclebams/hg38_wes/CDS-iKXYuH.hg38.bam CDS-sukIAT    gs://cclebams/hg38_wes/CDS-sukIAT.hg38.bam
Name: internal_bam_filepath, dtype: object
gs://cclebams/hg38_wes/CDS-7JWzyA.hg38.bam CDS-IAZz6r    gs://cclebams/hg38_wes/CDS-IAZz6r.hg38.bam
Name: internal_bam_filepath, dtype: object
gs://cclebams/hg38_wes/CDS-n7Fqfe.hg38.bam CDS-bweujf    gs://cclebams/hg38_wes/CDS-bweujf.hg38.bam
Name: internal_bam_filepath, dtype: object
gs://cclebams/hg38_wes/CDS-BRxHbu.hg38.bam CDS-zvvAOM    gs://cclebams/hg38_wes/CDS-zvvAOM.hg38.bam
Name: internal_bam_filepath, dtype: object
gs://cclebams/hg38_wes/CDS-nby0QM.hg38.bam CDS-Oy6cpj    gs://cclebams/hg38_wes/CDS-Oy6cpj.hg38.bam
Name: internal_bam_filepath, dtype: object
gs://cclebams/hg38_wes/CDS-2xSJmZ.hg38.bam CDS-0wO1Np    gs://cclebams/hg38_wes/CDS-0wO1Np.hg38.bam
Name: internal_bam_filepath, dtype: object
gs://cclebams/hg38_wes/CDS-5rD8XC.hg38.bam CDS-ML22GG    gs://cclebams/hg38_wes/CDS-ML22GG.hg38.bam
Name: internal_bam_filepath, dtype: object

In [13]:
## removing
gcp.rmFiles(torm, dryrun=False)

NameError: name 'torm' is not defined

In [72]:
## preventing remapping of found ones
weswm.update_sample_set("toremap", set(remapable.index) - set(found))

Sample set "toremap" (136 samples) successfully updated.


In [76]:
## are they very close to a related sample? (yes? drop)

[('gs://cclebams/hg38_wes/CDS-ydPJEM.hg38.bam', 33590349704),
 ('gs://cclebams/hg38_wes/CDS-s7pOQR.hg38.bam', 52259732517),
 ('gs://cclebams/wes/CDS-eGQYXr.bam', 24098037187),
 ('gs://cclebams/hg38_wes/CDS-Pkk9e2.hg38.bam', 16985791742),
 ('gs://cclebams/wes/CDS-Fz0HXE.bam', 25843558785),
 ('gs://cclebams/hg38_wes/CDS-soTPPi.hg38.bam', 15724674947),
 ('gs://cclebams/hg38_wes/CDS-iKXYuH.hg38.bam', 18181200052),
 ('gs://cclebams/wes/CDS-4S6juQ.bam', 29576566390),
 ('gs://cclebams/wes/CDS-UL1jLm.bam', 25516864764),
 ('gs://cclebams/rna/UPCISCC111.bam', 3821164738),
 ('gs://cclebams/hg38_wes/CDS-KgRznV.hg38.bam', 16089050631),
 ('gs://cclebams/hg38_wes/CDS-txTRwz.hg38.bam', 13238732111),
 ('gs://cclebams/wes/RP-1500/Exome/ESO26_1/current/ESO26_1.bam', 22776809217),
 ('gs://cclebams/hg38_wes/CDS-zwAn7G.hg38.bam', 20340862682),
 ('gs://cclebams/hg38_wes/CDS-fs8moU.hg38.bam', 21098867720),
 ('gs://cclebams/wes/CDS-gKIdjs.bam', 16134677521),
 ('gs://cclebams/hg38_wes/CDS-kAARUi.hg38.bam', 1877

In [ ]:
## can we find a related CDS id? (yes? find out if close else drop)
ccle_refsamples[ccle_refsamples.index.isin([i.split('/')[-1].split('.')[0] for i in remaining if "CDS" in i])]

In [78]:
## do we have this cell line already? (investigate, either add or drop)
sizze = [gcp.extractSize(u) for u in gcp.lsFiles([i for i in remaining if "CDS" not in i],'-l')]

listing files in gs


In [89]:
for val in sizze:
    print('------------')
    print(val)
    a = h.closest(ccle_refsamples['size'], val[1])
    print(a)
    print(ccle_refsamples[ccle_refsamples['size'] == a][['stripped_cell_name','arxspan_id']])

------------
('gs://cclebams/wes/RP-1500/Exome/ESO26_1/current/ESO26_1.bam', 22776809217)
22799628189.0
           stripped_cell_name  arxspan_id
CDS-nQIkC1            95T1000  ACH-001796
------------
('gs://cclebams/hg38_wes/ACH-001421_2.hg38.bam', 18871404272)
18871265251.0
           stripped_cell_name  arxspan_id
CDS-eyELts          RERFLCAD1  ACH-000791
------------
('gs://cclebams/wes/FARAGE.bam', 25369720311)
25349783889.0
           stripped_cell_name  arxspan_id
CDS-kNHP4Y             OV17R1  ACH-001373
------------
('gs://cclebams/wes/DepMap_CellLine_WES_May2019/RP-1561/Exome/P30OHK_1/v1/P30OHK_1.bam', 23761432371)
23743557362.0
           stripped_cell_name  arxspan_id
CDS-fAtTIx         NCCSTCK140  ACH-000110
------------
('gs://cclebams/wes/RP-1561/Exome/DOTC24510/current/DOTC24510.bam', 25448357213)
25349783889.0
           stripped_cell_name  arxspan_id
CDS-kNHP4Y             OV17R1  ACH-001373
------------
('gs://cclebams/wes/v2/WERIRB1.bam', 27335320368)
27163656004.0


20474056547.0
           stripped_cell_name  arxspan_id
CDS-TunXjE              U2940  ACH-001678
------------
('gs://cclebams/wes/RP-1561/Exome/Ca_Ski/current/Ca_Ski.bam', 21399623326)
21409812881.0
           stripped_cell_name  arxspan_id
CDS-SYO7e7                CJM  ACH-000458
------------
('gs://cclebams/wes/CCLE_DepMap_WES_July_Dec_2018/RP-1561/Exome/94T778/v1/94T778.bam', 25473572546)
25349783889.0
           stripped_cell_name  arxspan_id
CDS-kNHP4Y             OV17R1  ACH-001373
------------
('gs://cclebams/wes/RP-1561/Exome/TGBC1TKB/v1/TGBC1TKB.bam', 26496784045)
26348835512.0
           stripped_cell_name  arxspan_id
CDS-naUpJu                RH4  ACH-001765
------------
('gs://cclebams/wes/Getz_IBM_CellLine_WES_April2019/RP-1561/Exome/BPH1/v1/BPH1.bam', 24761705198)
24786017590.0
           stripped_cell_name  arxspan_id
CDS-j5EzNM            HCC1806  ACH-000624
------------
('gs://cclebams/wgs/G32831/Hs_944.T/v1/Hs_944.T.bam', 138381206764)
138735205256.0
           stri

In [ ]:
gs://cclebams/wes/RP-1500/Exome/ESO26_1/current/ESO26_1.bam'

In [91]:
for val in sizze:
    print('------------')
    print(val)
    a = h.closest(ccle_refsamples[~ccle_refsamples['legacy_size'].isna()]['legacy_size'], val[1])
    print(a)
    print(ccle_refsamples[ccle_refsamples['legacy_size'] == a][['stripped_cell_name','arxspan_id']])

------------
('gs://cclebams/wes/RP-1500/Exome/ESO26_1/current/ESO26_1.bam', 22776809217)
22787650593.0
           stripped_cell_name  arxspan_id
CDS-64cEne             LNZ308  ACH-000760
------------
('gs://cclebams/hg38_wes/ACH-001421_2.hg38.bam', 18871404272)
18870960791.0
           stripped_cell_name  arxspan_id
CDS-ChYgr0                EW7  ACH-002119
------------
('gs://cclebams/wes/FARAGE.bam', 25369720311)
25376759084.0
           stripped_cell_name  arxspan_id
CDS-YBCU7c           NCIH1436  ACH-000830
------------
('gs://cclebams/wes/DepMap_CellLine_WES_May2019/RP-1561/Exome/P30OHK_1/v1/P30OHK_1.bam', 23761432371)
23768902747.0
           stripped_cell_name  arxspan_id
CDS-Oez2VA           NCIBL128  ACH-002362
------------
('gs://cclebams/wes/RP-1561/Exome/DOTC24510/current/DOTC24510.bam', 25448357213)
25440914106.0
           stripped_cell_name  arxspan_id
CDS-ZzuUDl          TGBC52TKB  ACH-001862
------------
('gs://cclebams/wes/v2/WERIRB1.bam', 27335320368)
27327135877.0


21412825896.0
           stripped_cell_name  arxspan_id
CDS-rn0tZ5             UMUC11  ACH-001413
------------
('gs://cclebams/wes/CCLE_DepMap_WES_July_Dec_2018/RP-1561/Exome/94T778/v1/94T778.bam', 25473572546)
25465781084.0
           stripped_cell_name  arxspan_id
CDS-RM7bNy              LN319  ACH-000128
------------
('gs://cclebams/wes/RP-1561/Exome/TGBC1TKB/v1/TGBC1TKB.bam', 26496784045)
26491448378.0
           stripped_cell_name  arxspan_id
CDS-oejd1Z               C4II  ACH-001335
------------
('gs://cclebams/wes/Getz_IBM_CellLine_WES_April2019/RP-1561/Exome/BPH1/v1/BPH1.bam', 24761705198)
24759907179.0
           stripped_cell_name  arxspan_id
CDS-nymcCj               LAN2  ACH-001354
------------
('gs://cclebams/wgs/G32831/Hs_944.T/v1/Hs_944.T.bam', 138381206764)
138486776916.0
           stripped_cell_name  arxspan_id
CDS-rpqo0J             SNU475  ACH-000422
------------
('gs://cclebams/wes/RP-1561/Exome/TASK1/v1/TASK1.bam', 22239375545)
22238628098.0
           stripped_ce

In [101]:
## match md5 to bam files
md5s = [u for u in i if u.endswith('.md5')]
for val in md5s:
    rres = ccle_refsamples[(ccle_refsamples.internal_bam_filepath==val[:-4])&(ccle_refsamples.md5_hash.isna())]
    if len(rres)>0:
        print(rres)

In [102]:
## delete md5s
gcp.rmFiles(md5s, dryrun=False)

In [105]:
## missing tissue annot
set(ccle_refsamples[ccle_refsamples['primary_site'].isna() | ccle_refsamples['primary_disease'].isna() | ccle_refsamples['subtype'].isna() | ccle_refsamples['origin'].isna()].arxspan_id

,arxspan_id,version,sm_id,PDO,datatype,size,ccle_name,stripped_cell_name,participant_id,cellosaurus_id,...,19q2,19q3,19q4,20q1,20q2,20q4,20q,21Q1,low_quality,legacy_crc32_hash
CDS-55CRsR,ACH-001182,1,NaN,NaN,hybrid_capture,1.793164e+09,NaN,RPMI6666,PT-oFgMemwT,CVCL_1665,...,1.0,1.0,1.0,1.0,1.0,NaN,NaN,0,NaN,NaN
CDS-c7bFKt,ACH-000471,1,NaN,NaN,hybrid_capture,6.444237e+09,NaN,LI7,PT-vruNvPAR,CVCL_3840,...,1.0,1.0,1.0,1.0,1.0,NaN,NaN,0,NaN,NaN
CDS-5hu4Rw,ACH-001187,1,NaN,NaN,hybrid_capture,2.540050e+09,NaN,SF8657,PT-rocMG2rE,CVCL_K085,...,1.0,1.0,1.0,1.0,1.0,NaN,NaN,0,NaN,NaN
CDS-VjQVjt,ACH-001182,1,NaN,NaN,raindance,1.969057e+09,NaN,RPMI6666,PT-oFgMemwT,CVCL_1665,...,1.0,1.0,1.0,1.0,1.0,NaN,NaN,0,NaN,NaN
CDS-EclxvG,ACH-000471,1,NaN,NaN,raindance,1.504851e+09,NaN,LI7,PT-vruNvPAR,CVCL_3840,...,1.0,1.0,1.0,1.0,1.0,NaN,NaN,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CDS-J80Eb1,ACH-002034,1,NaN,PDO-21298,rna,4.940856e+09,NaN,T3M11,PT-hYmRJ4Oj,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN
CDS-Hb7hHv,ACH-002080,1,NaN,PDO-21298,rna,4.613839e+09,NaN,TN2,PT-h2eeF6an,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN
CDS-3VSkPT,ACH-002687,1,NaN,PDO-21569,rna,5.219565e+09,NaN,WM3772F,PT-MJutykcs,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN
CDS-2FgC1U,ACH-001707,1,NaN,"PDO-20722,PDO-21642",rna,7.056679e+09,NaN,WAOSEL,PT-y3RbI7uD,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN


In [106]:
## missing name/arxspan
ccle_refsamples[ccle_refsamples['arxspan_id'].isna() | ccle_refsamples['stripped_cell_name'].isna()]

,arxspan_id,version,sm_id,PDO,datatype,size,ccle_name,stripped_cell_name,participant_id,cellosaurus_id,...,19q2,19q3,19q4,20q1,20q2,20q4,20q,21Q1,low_quality,legacy_crc32_hash


In [107]:
# clean annots:
set(ccle_refsamples.primary_disease)

{'Adrenal Cancer',
 'Bile Duct Cancer',
 'Bladder Cancer',
 'Bone Cancer',
 'Brain Cancer',
 'Breast Cancer',
 'Cervical Cancer',
 'Colon/Colorectal Cancer',
 'Embryonal Cancer',
 'Endometrial/Uterine Cancer',
 'Engineered',
 'Esophageal Cancer',
 'Eye Cancer',
 'Gallbladder Cancer',
 'Gastric Cancer',
 'Head and Neck Cancer',
 'Kidney Cancer',
 'Leukemia',
 'Liposarcoma',
 'Liver Cancer',
 'Lung Cancer',
 'Lymphoma',
 'Myeloma',
 'Neuroblastoma',
 'Ovarian Cancer',
 'Pancreatic Cancer',
 'Prostate Cancer',
 'Rhabdoid',
 'Sarcoma',
 'Skin Cancer',
 'Teratoma',
 'Thymic Cancer',
 'Thyroid Cancer',
 'Unknown',
 'Vulvar',
 'esophagus_squamous',
 nan,
 'normal'}

In [108]:
set(ccle_refsamples.primary_site)

{'Colon',
 'Embryonal',
 'Esophageal Cancer',
 'Placenta',
 'Thymus',
 'abdomen',
 'ascites',
 'autonomic_ganglia',
 'biliary_tract',
 'bone',
 'bone_marrow',
 'brain',
 'breast',
 'central_nervous_system',
 'cervix',
 'endometrium',
 'engineered',
 'eye',
 'fibroblast',
 'haematopoietic_and_lymphoid_tissue',
 'kidney',
 'large_intestine',
 'liver',
 'lung',
 'lymph_node',
 nan,
 'oesophagus',
 'ovary',
 'pancreas',
 'pericardial_effusion',
 'pleura',
 'pleural_effusion',
 'prostate',
 'salivary_gland',
 'skin',
 'small_intestine',
 'soft_tissue',
 'spleen',
 'stomach',
 'thyroid',
 'upper_aerodigestive_tract',
 'urinary_tract',
 'uvea',
 'vul'}

In [109]:
set(ccle_refsamples.age)

{'0',
 '0.1',
 '0.25',
 '0.33',
 '0.5',
 '0.75',
 '0.8',
 '1',
 '1.4',
 '1.6',
 '1.67',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '2',
 '2.6',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '3',
 '3.5',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '4',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '5',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '6',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '68',
 '69',
 '7',
 '70',
 '71',
 '72',
 '73',
 '74',
 '75',
 '76',
 '77',
 '78',
 '79',
 '8',
 '80',
 '81',
 '82',
 '84',
 '85',
 '86',
 '87',
 '88',
 '89',
 '9',
 '92',
 'Adult',
 'C',
 'Children',
 'Embryo',
 'Fetus',
 'U',
 nan}

In [112]:
ccle_refsamples[ccle_refsamples.age.isna()]

,arxspan_id,version,sm_id,PDO,datatype,size,ccle_name,stripped_cell_name,participant_id,cellosaurus_id,...,19q2,19q3,19q4,20q1,20q2,20q4,20q,21Q1,low_quality,legacy_crc32_hash
CDS-yYeDem,ACH-001429,2,NaN,PDO-21569,rna,6.136561e+09,NaN,CCLFPEDS0009T,PT-gpiijlSj,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN
CDS-OT217E,ACH-001338,1,NaN,PDO-21298,rna,5.395430e+09,NaN,CHP134,PT-sqRdd0QF,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN
CDS-a8BCnW,ACH-001996,1,NaN,PDO-21298,rna,5.692887e+09,NaN,ICC18,PT-fZWks7bl,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN
CDS-zeFugQ,ACH-002707,1,NaN,PDO-21569,rna,4.253798e+09,NaN,IMSM2,PT-KQHkzAJs,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN
CDS-GOI63U,ACH-002077,1,NaN,PDO-21298,rna,4.736551e+09,NaN,LU165,PT-3za4R2YQ,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN
CDS-ldvG6N,ACH-001553,1,NaN,PDO-21298,rna,4.695594e+09,NaN,MEGA2,PT-SMrpbPt7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN
CDS-LFK9QA,ACH-002013,1,NaN,PDO-21298,rna,4.680192e+09,NaN,MM28,PT-ws3gMJ8I,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN
CDS-SdYhEt,ACH-002084,1,NaN,PDO-21298,rna,5.914711e+09,NaN,MMAC,PT-ViFHoQt0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN
CDS-gyyVs5,ACH-001992,1,NaN,PDO-21298,rna,6.628526e+09,NaN,ONE58,PT-77bcwWNt,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN
CDS-IgYH22,ACH-001633,1,NaN,PDO-21298,rna,5.966996e+09,NaN,PEO6,PT-kjmIMuN4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN


In [115]:
set(ccle_refsamples.subtype)

{'ALL',
 'AML',
 'ATL',
 'ATRT',
 'Acute Lymphoblastic Leukemia (ALL), B-cell',
 'Adenocarcinoma, clear cell',
 'B-cell',
 'B-cell, Non-Hodgkins, Burkitts',
 'Breast Ductal Carcinoma',
 'CLL',
 'CML',
 'Carcinoma',
 'Cholangiocarcinoma',
 'Engineered',
 'Ewing sarcoma',
 'Ewing_sarcoma',
 'Ewings Sarcoma',
 'Glioblastoma',
 'MMMT',
 'MPNST',
 'Medulloblastoma',
 'Melanoma',
 'Mesothelioma',
 'Mixed Uterine Adenocarcinoma and Rhabdomyosarcoma',
 'NSCLC',
 'Neuroblastoma',
 'Ovarian carcinoma',
 'PNET',
 'RMS',
 'SCCOHT',
 'SCLC',
 'Small Cell Lung Cancer (SCLC)',
 'Squamous Cell Carcinoma',
 'Testicular',
 'Uveal Melanoma',
 'acute_biphenotypic_leukemia',
 'adrenal_carcinoma',
 'bladder_carcinoma',
 'breast_adenocarcinoma',
 'breast_carcinoma',
 'breast_ductal_carcinoma',
 'brenner_tumor',
 'caecum_adenocarcinoma',
 'cervical_adenocarcinoma',
 'cervical_carcinoma',
 'cervical_squamous',
 'cholangiocarcinoma',
 'chondrosarcoma',
 'chordoma',
 'choriocarcinoma',
 'clear_cell_carcinoma',
 

In [119]:
set(ccle_refsamples.origin)

{'adrenal_cortex',
 'bile_duct',
 'bladder',
 'blood',
 'bone',
 'breast',
 'central_nervous_system',
 'cervix',
 'colorectal',
 'embryo',
 'engineered',
 'engineered_blood',
 'engineered_breast',
 'engineered_central_nervous_system',
 'engineered_kidney',
 'engineered_lung',
 'engineered_ovary',
 'engineered_prostate',
 'engineered_soft_tissue',
 'esophagus',
 'eye',
 'fibroblast',
 'gastric',
 'kidney',
 'liver',
 'lung',
 'lymphocyte',
 nan,
 'ovary',
 'pancreas',
 'peripheral_nervous_system',
 'plasma_cell',
 'prostate',
 'skin',
 'soft_tissue',
 'thyroid',
 'unknown',
 'upper_aerodigestive',
 'urinary_tract',
 'uterus'}

In [118]:
set(ccle_refsamples.columns)

{'18q1',
 '18q2',
 '18q3',
 '18q4',
 '19q1',
 '19q2',
 '19q3',
 '19q4',
 '20q',
 '20q1',
 '20q2',
 '20q4',
 '21Q1',
 'PDO',
 'age',
 'arxspan_id',
 'baits',
 'bam_public_sra_path',
 'bam_qc',
 'ccle_name',
 'cellosaurus_id',
 'comments',
 'condition',
 'crc32c_hash',
 'datatype',
 'internal_bai_filepath',
 'internal_bam_filepath',
 'legacy_bai_filepath',
 'legacy_bam_filepath',
 'legacy_crc32_hash',
 'legacy_crc32c_hash',
 'legacy_size',
 'low_quality',
 'matched_normal',
 'md5_hash',
 'mediatype',
 'origin',
 'parent_cell_line',
 'participant_id',
 'primary_disease',
 'primary_site',
 'processing_qc',
 'release_date',
 'sequencing_date',
 'sequencing_type',
 'sex',
 'size',
 'sm_id',
 'source',
 'stripped_cell_name',
 'subsubtype',
 'subtype',
 'update_time',
 'version'}

In [114]:
ccle_refsamples[ccle_refsamples.source.isna()]

,arxspan_id,version,sm_id,PDO,datatype,size,ccle_name,stripped_cell_name,participant_id,cellosaurus_id,...,19q2,19q3,19q4,20q1,20q2,20q4,20q,21Q1,low_quality,legacy_crc32_hash
CDS-6Je1tY,ACH-001605,2,NaN,NaN,rna,4.654329e+09,NaN,NO10,PT-oZhLPsoJ,CVCL_3075,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN
CDS-57hTZY,ACH-001046,1,NaN,NaN,rna,7.399863e+09,NaN,CORL26,PT-Z2Fq3yte,CVCL_2410,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN
CDS-j3NCLS,ACH-002047,1,NaN,NaN,rna,4.500937e+09,NaN,HIRSBM,PT-OOZstzSA,CVCL_E060,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN
CDS-FMkkVj,ACH-001648,1,NaN,NaN,rna,4.050264e+09,NaN,SHMAC4,PT-oBrDoJPm,U,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN
CDS-6HLVzC,ACH-001227,1,NaN,NaN,rna,5.268985e+09,NaN,UPCISCC090,PT-mkNL4JuM,U,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CDS-PoGjEx,ACH-001641,1,NaN,PDO-21298,rna,7.813216e+09,NaN,SAT,PT-uOqhQJIC,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN
CDS-J80Eb1,ACH-002034,1,NaN,PDO-21298,rna,4.940856e+09,NaN,T3M11,PT-hYmRJ4Oj,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN
CDS-Hb7hHv,ACH-002080,1,NaN,PDO-21298,rna,4.613839e+09,NaN,TN2,PT-h2eeF6an,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN
CDS-3VSkPT,ACH-002687,1,NaN,PDO-21569,rna,5.219565e+09,NaN,WM3772F,PT-MJutykcs,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN


In [111]:
ccle_refsamples[ccle_refsamples.sex.isna()]

,arxspan_id,version,sm_id,PDO,datatype,size,ccle_name,stripped_cell_name,participant_id,cellosaurus_id,...,19q2,19q3,19q4,20q1,20q2,20q4,20q,21Q1,low_quality,legacy_crc32_hash
CDS-yYeDem,ACH-001429,2,NaN,PDO-21569,rna,6.136561e+09,NaN,CCLFPEDS0009T,PT-gpiijlSj,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN
CDS-OT217E,ACH-001338,1,NaN,PDO-21298,rna,5.395430e+09,NaN,CHP134,PT-sqRdd0QF,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN
CDS-a8BCnW,ACH-001996,1,NaN,PDO-21298,rna,5.692887e+09,NaN,ICC18,PT-fZWks7bl,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN
CDS-zeFugQ,ACH-002707,1,NaN,PDO-21569,rna,4.253798e+09,NaN,IMSM2,PT-KQHkzAJs,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN
CDS-GOI63U,ACH-002077,1,NaN,PDO-21298,rna,4.736551e+09,NaN,LU165,PT-3za4R2YQ,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN
CDS-ldvG6N,ACH-001553,1,NaN,PDO-21298,rna,4.695594e+09,NaN,MEGA2,PT-SMrpbPt7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN
CDS-LFK9QA,ACH-002013,1,NaN,PDO-21298,rna,4.680192e+09,NaN,MM28,PT-ws3gMJ8I,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN
CDS-SdYhEt,ACH-002084,1,NaN,PDO-21298,rna,5.914711e+09,NaN,MMAC,PT-ViFHoQt0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN
CDS-gyyVs5,ACH-001992,1,NaN,PDO-21298,rna,6.628526e+09,NaN,ONE58,PT-77bcwWNt,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN
CDS-IgYH22,ACH-001633,1,NaN,PDO-21298,rna,5.966996e+09,NaN,PEO6,PT-kjmIMuN4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,NaN


In [261]:
wm.disable_hound().update_sample_attributes(a)

Successfully updated attributes '['legacy_bam_filepath', 'legacy_bai_filepath', 'legacy_size', 'legacy_crc32c_hash', 'internal_bam_filepath', 'internal_bam_filepath', 'size', 'crc32c_hash', 'md5_hash']' for 412 samples.


In [262]:
wm = dm.WorkspaceManager('broad-firecloud-ccle/DepMap_hg38_RNAseq')

In [490]:
a = wm.get_samples()

In [300]:
ss= h.dups(ccle_refsamples['size'].tolist()+ccle_refsamples['legacy_size'].tolist())

In [307]:
issue = {val:[] for val in ss}
recom = []
for val in ss:
    for k,v in ccle_refsamples[(ccle_refsamples['size']==val)].iterrows():
        if v.size!= gcp.extractSize(gcp.lsFiles([v.internal_bam_filepath],"-l")[0]):
            recom.append(k)
        else:
            issue[val].append(k)
    for k,v in ccle_refsamples[(ccle_refsamples['legacy_size']==val)].iterrows():
        if v.legacy_size!= gcp.extractSize(gcp.lsFiles([v.legacy_bam_filepath],"-l")[0]):
            recom.append(k)    
        else:
            issue[val].append(k)

listing files in gs
listing files in gs
listing files in gs
listing files in gs
listing files in gs
listing files in gs
listing files in gs
listing files in gs
listing files in gs
listing files in gs
listing files in gs
listing files in gs
listing files in gs
listing files in gs


KeyboardInterrupt: 

In [422]:
e = gcp.lsFiles(a.legacy_bam_filepath.tolist(),"-L")

listing files in gs


In [418]:
wm.update_sample_attributes(a)

Successfully updated attributes '['legacy_bam_filepath', 'legacy_bai_filepath', 'legacy_size', 'legacy_crc32c_hash']' for 15 samples.
Hound executing batch upload of 150 records


In [321]:
for k, val in ccle_refsamples[(ccle_refsamples.datatype=='rna')&(~ccle_refsamples.internal_bam_filepath.str.contains("/rnasq_hg38",na=True))].internal_bam_filepath.iteritems():
    rn = "gs://cclebams/rna/"+'/'.join(val.split('/')[3:])
    print(rn)
    rni = rn[:-1]+'i'
    vali = val[:-1]+"i"
    #! gsutil cp $val $rn
    #! gsutil cp $vali $rni
    ccle_refsamples.loc[k, "internal_bam_filepath"] = rn
    ccle_refsamples.loc[k, "internal_bai_filepath"] = rni
    ccle_refsamples.loc[k, "size"] = gcp.extractSize(gcp.lsFiles([rn], '-l')[0],)[1]
    ccle_refsamples.loc[k, "crc32c_hash"] = gcp.extractHash(gcp.lsFiles([rn], '-L')[0])[1]
    ccle_refsamples.loc[k, "sequencing_date"]= seq.getBamDate([rn])[0]
    print(rn)
    break

gs://cclebams/rna/rna/rna/rna/rna/DepMap_CellLine_RNAseq_Feb2020/RP-1561/RNA/ONDA10/v1/ONDA10.bam
Copying gs://cclebams/rna/rna/rna/rna/DepMap_CellLine_RNAseq_Feb2020/RP-1561/RNA/ONDA10/v1/ONDA10.bam [Content-Type=application/octet-stream]...
/ [1 files][  4.8 GiB/  4.8 GiB]                                                
Operation completed over 1 objects/4.8 GiB.                                      
Copying gs://cclebams/rna/rna/rna/rna/DepMap_CellLine_RNAseq_Feb2020/RP-1561/RNA/ONDA10/v1/ONDA10.bai [Content-Type=application/octet-stream]...
/ [1 files][  6.5 MiB/  6.5 MiB]                                                
Operation completed over 1 objects/6.5 MiB.                                      
listing files in gs
listing files in gs
gs://cclebams/rna/rna/rna/rna/rna/DepMap_CellLine_RNAseq_Feb2020/RP-1561/RNA/ONDA10/v1/ONDA10.bam
gs://cclebams/rna/rna/Getz_IBM_CellLine_RNAseq_June2020/RP-1561/RNA/CORL26/v1/CORL26.bam
Copying gs://cclebams/rna/Getz_IBM_CellLine_RNAseq_June2020

KeyboardInterrupt: 

In [377]:
ccle_refsamples.loc[a[a.legacy_bam_filepath.isna()].index].internal_bam_filepath

sample_id
CDS-2FgC1U    gs://cclebams/rnasq_hg38/CDS-2FgC1U.Aligned.so...
CDS-3VSkPT    gs://cclebams/rnasq_hg38/CDS-3VSkPT.Aligned.so...
CDS-57hTZY    gs://cclebams/rnasq_hg38/CDS-57hTZY.Aligned.so...
CDS-6HLVzC    gs://cclebams/rnasq_hg38/CDS-6HLVzC.Aligned.so...
CDS-6Je1tY    gs://cclebams/rnasq_hg38/CDS-6Je1tY.Aligned.so...
CDS-a8BCnW    gs://cclebams/rnasq_hg38/CDS-a8BCnW.Aligned.so...
CDS-bmH4HU    gs://cclebams/rnasq_hg38/CDS-bmH4HU.Aligned.so...
CDS-FMkkVj    gs://cclebams/rnasq_hg38/CDS-FMkkVj.Aligned.so...
CDS-GOI63U    gs://cclebams/rnasq_hg38/CDS-GOI63U.Aligned.so...
CDS-gyyVs5    gs://cclebams/rnasq_hg38/CDS-gyyVs5.Aligned.so...
CDS-Hb7hHv    gs://cclebams/rnasq_hg38/CDS-Hb7hHv.Aligned.so...
CDS-IgYH22    gs://cclebams/rnasq_hg38/CDS-IgYH22.Aligned.so...
CDS-j3NCLS    gs://cclebams/rnasq_hg38/CDS-j3NCLS.Aligned.so...
CDS-J80Eb1    gs://cclebams/rnasq_hg38/CDS-J80Eb1.Aligned.so...
CDS-ldvG6N    gs://cclebams/rnasq_hg38/CDS-ldvG6N.Aligned.so...
CDS-LFK9QA    gs://cclebams/rn

In [212]:
ccle_refsamples.loc[ccle_refsamples[ccle_refsamples.datatype.isin(['rna'])].index.tolist(),'size']

CDS-6dPoX7    13488934939
CDS-zmiamL     7606219711
CDS-3wjudI     8571265034
CDS-uUP01E     8733556513
CDS-UFAuQl     8852095963
                 ...     
CDS-PoGjEx     7813216031
CDS-J80Eb1     4940855845
CDS-Hb7hHv     4613838702
CDS-3VSkPT     5219564814
CDS-2FgC1U     7056679221
Name: size, Length: 1516, dtype: int64

In [225]:
ccle_refsamples.loc[ccle_refsamples[ccle_refsamples.datatype.isin(['rna'])].index.tolist(),'size'] = [gcp.extractSize(i)[1] for i in gcp.lsFiles(ccle_refsamples[ccle_refsamples.datatype.isin(['rna'])]['internal_bam_filepath'].tolist(),'-l')]

ccle_refsamples.loc[ccle_refsamples[ccle_refsamples.datatype.isin(['rna'])].index.tolist(),'crc32c_hash'] = [gcp.extractHash(i) for i in gcp.lsFiles(ccle_refsamples[ccle_refsamples.datatype.isin(['rna'])]['internal_bam_filepath'].tolist(),'-L')]

listing files in gs
listing files in gs


In [156]:
ccle_refsamples[ccle_refsamples.datatype.isin(['wgs'])][["internal_bam_filepath", "internal_bai_filepath", "md5_hash","legacy_bam_filepath","legacy_bai_filepath",'legacy_size', 'legacy_crc32c_hash']]

,internal_bam_filepath,internal_bai_filepath,md5_hash,legacy_bam_filepath,legacy_bai_filepath,legacy_size,legacy_crc32c_hash
CDS-uhGMmZ,gs://cclebams/wgs_hg38/CDS-uhGMmZ.wgs_ccle.bam,gs://cclebams/wgs_hg38/CDS-uhGMmZ.wgs_ccle.bai,6136551f350a8e8b6b2ca478ac951d49,gs://cclebams/wgs/G16444/G-401/v1/G-401.bam,gs://cclebams/wgs/G16444/G-401/v1/G-401.bai,1.834687e+11,puzdUg==
CDS-vNNLAr,gs://cclebams/wgs_hg38/CDS-vNNLAr.wgs_ccle.bam,gs://cclebams/wgs_hg38/CDS-vNNLAr.wgs_ccle.bai,470c011136c545e5eea12cf71a2910c9,gs://cclebams/wgs/G32831/G-402/v2/G-402.bam,gs://cclebams/wgs/G32831/G-402/v2/G-402.bai,1.428883e+11,08KlNA==
CDS-KoNTMz,gs://cclebams/wgs_hg38/CDS-KoNTMz.wgs_ccle.bam,gs://cclebams/wgs_hg38/CDS-KoNTMz.wgs_ccle.bai,9d00ef2ecef3854d16d3693733760507,gs://cclebams/wgs/G16449/CHP-212/v2/CHP-212.bam,gs://cclebams/wgs/G16449/CHP-212/v2/CHP-212.bai,1.651190e+11,NmAKsw==
CDS-Wmm2y2,gs://cclebams/wgs_hg38/CDS-Wmm2y2.wgs_ccle.bam,gs://cclebams/wgs_hg38/CDS-Wmm2y2.wgs_ccle.bai,f81251027a48bf0dd030260c1b2d7ebf,gs://cclebams/wgs/G32831/KYM-1/v2/KYM-1.bam,gs://cclebams/wgs/G32831/KYM-1/v2/KYM-1.bai,1.860679e+11,nQhYUA==
CDS-IKutdC,gs://cclebams/wgs_hg38/CDS-IKutdC.wgs_ccle.bam,gs://cclebams/wgs_hg38/CDS-IKutdC.wgs_ccle.bai,de1720234eaa10b3ee38fa69cc4f0649,gs://cclebams/wgs/G32831/A-204/v3/A-204.bam,gs://cclebams/wgs/G32831/A-204/v3/A-204.bai,1.513434e+11,c1qSGg==
...,...,...,...,...,...,...,...
CDS-lEoO4A,gs://cclebams/wgs/CDS-lEoO4A.bam,gs://cclebams/wgs/CDS-lEoO4A.bai,c5368347afae1031645835742185b799,gs://cclebams/wgs/CDS-lEoO4A.bam,gs://cclebams/wgs/CDS-lEoO4A.bai,8.854451e+10,tFgKDw==
CDS-lZD981,gs://cclebams/wgs/CDS-lZD981.bam,gs://cclebams/wgs/CDS-lZD981.bai,35e4b81c789393791ad5c1e2c70cfd1b,gs://cclebams/wgs/CDS-lZD981.bam,gs://cclebams/wgs/CDS-lZD981.bai,7.308927e+10,sbmvbA==
CDS-M9kJ4N,gs://cclebams/wgs/CDS-M9kJ4N.bam,gs://cclebams/wgs/CDS-M9kJ4N.bai,eb51988decdf6e53473006d833d4fce0,gs://cclebams/wgs/CDS-M9kJ4N.bam,gs://cclebams/wgs/CDS-M9kJ4N.bai,7.108076e+10,LdAyfw==
CDS-bNHw6E,gs://cclebams/wgs/CDS-bNHw6E.bam,gs://cclebams/wgs/CDS-bNHw6E.bai,c1cb0522c5ba22df53e1b684e6f12a64,gs://cclebams/wgs/CDS-bNHw6E.bam,gs://cclebams/wgs/CDS-bNHw6E.bai,7.505758e+10,NHvcFg==
